<a href="https://colab.research.google.com/github/chandrusuresh/CarND-Path-Planning-Project/blob/master/PathPlanningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Path Planning Project

In this project, a demonstration of a path planning algorithm is presented to help a vehicle navigate a busy highway as fast as possible within the speed limit of 50mph and as safe as possible without any collisions. There are 3 lanes on the highway and the path planning algorithm needs to choose an appropriate lane and speed for navigation. This demonstration is based on a simulator developed by Udacity and was completed as part of Udacity's Self-Driving Car Nanodegree. The [Udacity github repo](https://github.com/udacity/CarND-Path-Planning-Project) for this project has all the details about the software used in the simulator and the installation instructions.

### Summary of Setup Instructions
1. The project uses [uWebSocketIO](https://github.com/uNetworking/uWebSockets) for communication between the user-written algorithm and the simulator. Udacity has provided bash scripts to install this in both [Linux](https://github.com/udacity/CarND-Path-Planning-Project/blob/master/install-ubuntu.sh)/[Mac](https://github.com/udacity/CarND-Path-Planning-Project/blob/master/install-mac.sh) environments. These scripts are included in this repo.
2. The simulator can be downloaded from [here](https://github.com/udacity/self-driving-car-sim/releases/tag/T3_v1.2).
3. To run the simulator on Mac/Linux, first make the binary file executable with the following: `sudo chmod u+x term3_sim.x86_64`.

### Basic Build Instructions (Linux/Mac)
1. Clone this repo.
2. Make a build directory: `mkdir build && cd build`
3. Compile: `cmake .. && make`
4. Run it: `./path_planning`
This should setup a listener for the code to get data from the simulator.
5. Launch the simulator from a terminal: `./term3_sim.x86_64`.
6. Select the Path Planning project and click start to start the simulation.

These steps should get any user in a Linux/Mac environment up and running with the code.

## Goals & Problem Specifications
The goal of this project is to develop a path planning algorithm to safely navigate a car around a virtual highway with other traffic that is driving ±10 MPH of the 50 MPH speed limit. The car's localization and sensor fusion data will be provided along with a sparse map list of waypoints around the highway. The car is supposed to maintain speeds as close as possible to the 50 MPH speed limit, which means passing slower traffic when possible. The algorithm is to avoid collision with other cars and keep the car within the marked road lanes at all times, unless during lane change maneuvers. The final goal is for the car to complete one loop around the 6946m highway (~5 minutes at 50 MPH) with acceleration and jerk strictly less than over 10 ms^-2^ & 10 ms^-2^ respectively.

## Map
The map of the highway is in `data/highway_map.txt`

Each waypoint in the list contains [x,y,s,dx,dy] values. x and y are the waypoint's map coordinate position, the s value is the distance along the road to get to that waypoint in meters, the dx and dy values define the unit normal vector pointing outward of the highway loop.

The highway's waypoints loop around so the frenet s value, distance along the road, goes from 0 to 6945.554.

## Localization
Main car's localization Data (No Noise) is avilable as:
*   ["x"]     The car's x position in map coordinates
*   ["y"]     The car's y position in map coordinates
*   ["s"]     The car's s position in frenet coordinates
*   ["d"]     The car's d position in frenet coordinates
*   ["yaw"]   The car's yaw angle in the map
*   ["speed"] The car's speed in MPH

## Sensor Fusion
Sensor fusion data containing a list of all other car's (noise-free) attributes on the same side of the road is available as the following 2D vector.

["sensor_fusion"] A 2d vector of cars
Each item in the 2d vector is a vector containing the following info of each nearby car:
*   unique ID
*   x position in map coordinates
*   y position in map coordinates
*   x velocity in m/s
*   y velocity in m/s
*   Distance along road (s position in frenet coordinates)
*   Cross-track error (d position in frenet coordinates)

## Other Implementation Details
*   The car uses a perfect controller and will visit every (x,y) point in the trajectory list every .02 seconds. The units for the (x,y) points are in meters and the spacing of the points determines the speed of the car. The vector going from a point to the next point in the list dictates the angle of the car. Acceleration both in the tangential and normal directions is measured along with the jerk (the rate of change of total acceleration). The (x,y) point paths that the planner recieves should not have a total acceleration that goes over 10 m/s^2, also the jerk should not go over 50 m/s^3. (NOTE: As this is BETA, these requirements might change. Also currently jerk is over a .02 second interval, it would probably be better to average total acceleration over 1 second and measure jerk from that.

*   There will be some latency between the simulator running and the path planner returning a path, with optimized code usually its not very long maybe just 1-3 time steps. During this delay the simulator will continue using points that it was last given, because of this its a good idea to store the last points you have used so you can have a smooth transition. previous_path_x, and previous_path_y can be helpful for this transition since they show the last points given to the simulator controller with the processed points already removed. You would either return a path that extends this previous path or make sure to create a new path that has a smooth transition with this last path.


## Details

### Finite State Machine
Given that the highway to navigate has a fixed number of lanes (= 3), the following states are defined for the finite state machine.
*   Keep Lane (KL) - This state essentially finds a trajectory that keeps the car within the current lane. The car is initialized at the start of the simulation to this state.
*   Planning Lane Change Left/Right (PLCL/PLCR) - This state maintains the KL trajectory while adjusting the speed of the car to prepare the car for a lane change maneuver. The lane change is executed as soon as the safety and speed constraints are satisfied. Details of these conditions are described in subsequent sections.
*   Lane Change Left/Right (LCL/LCR) - This state corresponds to when the car is executing the lane change maneuver. This is the only state where the car is allowed to cross between lanes.
*   (Optional) Abort Lane Change- This state is critical for safety requirement, but is not an explicit part of this FSM. The safety and speed constraints mentioned above are constantly checked during the LCL/LCR states and when violated will change the state to KL. This state is specified here for the sake of completeness, but will not be discussed further.





### Algorithm
1. Initialize state to `KL`. Set Reference Velocity `ref_vel = 0`
2. Get all cars within threshold `range=100m` of ego car.
3. Find the closest cars ahead and behind of the ego car in each lane and get their speeds.
4. Check if lane change is feasible by comparing the speeds & ranges of closest cars in neighboring lanes.
> 1. A lane change criteria is set based on the current FSM state as follows:
>> 1. In `KL`: `laneChangeCriteria=20 m`
>> 2. In `PLCL/PLCR`: `laneChangeCriteria=15 m`
>> 3. In `LCL/LCR`: `laneChangeCriteria=10 m`
> 2. In each lane, $i$:
>> 1. Get the states of the closest vehicle ahead  (indexed by $j$) and behind  ($k$) the ego car.
>> 2. If both the cars $j$ & $k$ are within `laneChangeCriteria`, lane change is not feasible. Set $`PLC[i]=false`.
>> 3. If `PLC[i] != false`, compute the following metric, $\delta_i$:
$$ \begin{align} \Delta s_i = s_{j,i} - s_e \\ \Delta v_i = v_e - v_{j,i} \\ \delta_i = \frac{\Delta s_i}{\Delta v_i} - \frac{\Delta v_i}{a_{\max}}\end{align}$$  where $a_{\max}$ is acceleration limit.
>> 4. If $\delta_i > 0$ for both the cars $j$ & $k$, lane change is feasible. Set `PLC[i]=true`.
5. We compute reference speeds for ego car for each lane $i$ based on the speeds of the closest cars $j$ & $k$  
> 1. For the closest car ahead: $v1_{i} = \max(v_{j,i},v_e - a_{\max}\Delta t)$
> 2. For the closest car behind: $v2_{i} = v_{k,i}$

>>>> $v_{ref,i} = \min(v_e + a_{\max}\Delta t,v1_i,v2_i,\text{Speed Limit})$

6. Trajectories from current lane to the middle of every other lane $i$ are generated by interpolating a spline from current position to the middle of each lane for a distance `100 m` ahead.
7. Each of the trajectories are evaluated against a cost metric to pick the best next lane (and FSM state). 
> 1. Speed Cost: A speed cost function is for each lane determines a metric that takes into account the difference of speed from the speed limit and how close each car is to the ego car for each lane.
$$ \text{Speed Cost}(i) = \frac{1}{n_i}\sum_{p=1}^{n_i}{\frac{\text{Speed Limit} - v_{p,i}}{\lvert s_e - s_{p,i} \rvert}} $$
> 2. Lane Change Cost: To prevent constant switching between lanes, a lane change penalty of 0.5 if applied as a factor to the speed cost function value for each lane change. For each lane $i$:
>> * If $i == \text{current lane}$: `Lane change cost(i) = 1.0`
>> * If $i \ne \text{current lane}$ & `PLC[i] = true`: `Lane change cost(i) = 0.5`
>> * Else: `Lane change cost(i) = 1000` since lane change is not feasible.
> 3. $\text{Total Cost(i)} = \text{Lane change cost(i)} \times \text{Speed Cost}(i)$
8. From the total costs, the next FSM state is determined by:
> 1. If `current state = KL` & $\arg\min(\text{Total Cost}) == \text{current lane}$, `next_state = KL`
> 2. If `current state = KL` & $\arg\min(\text{Total Cost}) < \text{current lane}$, `next_state = PLCL`
> 3. If `current state = KL` & $\arg\min(\text{Total Cost}) > \text{current lane}$, `next_state = PLCR`


This scheme supports a double lane change maneuver as a sequence of multiple single lane change maneuvers. The algorithm also supports aborting lane changes either in the `PLCL/PLCR` or `LCL/LCR` states.








### Map & Coordinate Systems
The `map_waypoints_s`,`map_waypoints_x` & `map_waypoints_y` are 1D vectors containing the frenet-s (distance from start of loop), global X and global 
Y coordinate of the highway.

### Car States
The state of each car is defined by a vector of size 6 defined by:
*   unique ID (-1 for ego car)
*   Lane ID
*   Distance along road (s position in frenet coordinates)
*   Cross-track error (d position in frenet coordinates)
*   x velocity in m/s
*   y velocity in m/s

### Get Closest Cars
A threshold for tracking closest cars is set to 100m. From the sensor fusion data, all the cars that are within this range of the ego-car are determined and their motion predicted for the next time step. Based on this prediction, the lane location of these nearby cars is determined.